# Spotify Playlist Personalisation

----

### 1. Set Environmental variables

Use auth_example.env as a sample. Provide all 3 secrets set in your spotify developer platform

* SPOTIFY_CLIENT_ID=
* SPOTIFY_CLIENT_SECRET=
* SPOTIFY_REDIRECT_URI=


In [ ]:
from dotenv import load_dotenv

# Load .env file
load_dotenv()


### 2. Generate Token
Provide call back link to generate token for use

In [ ]:
import tekore as tk
conf = tk.config_from_environment()
token = tk.prompt_for_user_token(*conf)


### 3. Retrieving Playlist & Track Data

Reference util.py for additional functionality for data pull.

Utilising tekore package to interact with Spotify API.


In [ ]:
playlistTracks = []
tasks = [get_playlist_tracks_ids(async_spotify, id) for id in playlist_ids[:5]]
responses = await asyncio.gather(*tasks)
for response in responses:
    playlistTracks.extend(response)

In [ ]:
playlist_ids = [playlist.id for playlist in a]

In [ ]:
track_ids = await get_playlist_tracks_ids(async_spotify, playlistID='0owDaOLnhuRaxuzClvLGEP')

In [ ]:
track_features = await get_tracks_audio_features(async_spotify, tracks = track_ids)
track_features = [track.__dict__ for track in track_features if track is not None]


In [ ]:
import dask.dataframe as dd
import pandas as pd

In [ ]:
df = pd.DataFrame(track_features)
df.head(5)

In [ ]:
features = [
    'acousticness', 
    'danceability',
    'energy',
    'instrumentalness',
    'key',
    'liveness',
    'loudness',
    'mode',
    'speechiness',
    'tempo',
    'time_signature',
    'valence'
]

In [ ]:
track_features[0]


In [ ]:
from sklearn.decomposition import PCA
import seaborn as sns
import matplotlib.pyplot as plt
import seaborn as sns

# Define PCA object
pca = PCA(n_components=2)

# Apply PCA
pca_result = pca.fit_transform(df[features].values)

# Create a DataFrame with the results
df_pca = pd.DataFrame(data = pca_result, columns = ['PCA 1', 'PCA 2'])

# Plot the results
sns.scatterplot(x="PCA 1", y="PCA 2", data=df_pca)
plt.show()


In [ ]:
from sklearn.manifold import TSNE

# Define TSNE object
tsne = TSNE(n_components=2)

# Apply TSNE
tsne_result = tsne.fit_transform(df[features].values)

# Create a DataFrame with the results
df_tsne = pd.DataFrame(data = tsne_result, columns = ['TSNE 1', 'TSNE 2'])

# Plot the results
sns.scatterplot(x="TSNE 1", y="TSNE 2", data=df_tsne)
plt.show()
